<a href="https://colab.research.google.com/github/GrandMoff100/ML-collab-notebooks/blob/main/SubFeatureSamplingKMeans.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import random
import math
from sklearn.cluster import KMeans
import itertools

df = pd.read_csv("https://raw.githubusercontent.com/dvescelius/MLCamp2022/main/wine.csv")


def cmatrix_score(cm):
    return cm[0][0] + cm[1][1]


features = [
    "fixed acidity",
    "volatile acidity",
    "citric acid",
    "residual sugar",
    "chlorides",
    "free sulfur dioxide",
    "total sulfur dioxide",
    "density",
    "pH",
    "sulphates",
    "alcohol",
    "quality"
]


def k2_means(reduced_features):
    cluster_df = df[list(reduced_features)]
    model = KMeans(n_clusters=2, random_state=24)
    model.fit(cluster_df)
    model_df = cluster_df.copy(deep=True)
    model_df['Cluster'] = model.labels_
    ct = pd.crosstab(model_df["Cluster"], df["type"])
    score = cmatrix_score(ct.values) / len(model_df)
    return score, ct.values, model, reduced_features


def iterate_features(all_features):
    for count in range(2, len(all_features)):
        total_combos = math.factorial(len(all_features))
        total_combos //= math.factorial(count)
        total_combos //= math.factorial(len(all_features) - count)
        for i, feature_set in enumerate(itertools.combinations(all_features, count)):
            score, _, _, feature_set = result = k2_means(feature_set)
            print(f"F{count}: [{i + 1}/{total_combos}] {score} {feature_set}")
            yield result


def sift_features(all_features):
    scores, confusion_matrices, models, list_of_feature_sets = zip(*iterate_features(all_features))
    for model, feature_set, score in zip(models, confusion_matrices, list_of_feature_sets, scores):
        if score == max(scores):
            yield model, feature_set, score

In [ ]:
sifted_features = tuple(sift_features(features))

# Most Accurate Feature Selection is
# `F3: [92/220] 0.9107280283207634 ('volatile acidity', 'density', 'sulphates')`
    

F2: [1/66] 0.187932892104048 ('fixed acidity', 'volatile acidity')
F2: [2/66] 0.8014468216099738 ('fixed acidity', 'citric acid')
F2: [3/66] 0.5317839002616592 ('fixed acidity', 'residual sugar')
F2: [4/66] 0.19855317839002617 ('fixed acidity', 'chlorides')
F2: [5/66] 0.609819916884716 ('fixed acidity', 'free sulfur dioxide')
F2: [6/66] 0.7951362167154071 ('fixed acidity', 'total sulfur dioxide')
F2: [7/66] 0.19855317839002617 ('fixed acidity', 'density')
F2: [8/66] 0.19855317839002617 ('fixed acidity', 'pH')
F2: [9/66] 0.18777897491149761 ('fixed acidity', 'sulphates')
F2: [10/66] 0.48360781899338157 ('fixed acidity', 'alcohol')
F2: [11/66] 0.18700938894874558 ('fixed acidity', 'quality')
F2: [12/66] 0.14560566415268586 ('volatile acidity', 'citric acid')
F2: [13/66] 0.5293212251808527 ('volatile acidity', 'residual sugar')
F2: [14/66] 0.13606279821456058 ('volatile acidity', 'chlorides')
F2: [15/66] 0.609819916884716 ('volatile acidity', 'free sulfur dioxide')
F2: [16/66] 0.795136216

ValueError: ignored